In [1]:
from cryptography import x509
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
import datetime

In [61]:
def create_ca(common_name: str):
    private_key_ca = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    
    x509_common_name = x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)
    
    subject_name_ca = x509.Name([x509_common_name])
    issuer_ca = x509.Name([x509_common_name])
    
    cert_ca = x509.CertificateBuilder() \
        .subject_name(subject_name_ca) \
        .issuer_name(issuer_ca) \
        .public_key(private_key_ca.public_key()) \
        .serial_number(x509.random_serial_number()) \
        .not_valid_before(datetime.datetime.utcnow()) \
        .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=3650)) \
        .sign(private_key_ca, hashes.SHA256())

    return cert_ca, private_key_ca


def create_csr(common_name: str, email: str=None):
    # Ustvarimo zasebni ključ
    private_key_csr = rsa.generate_private_key(public_exponent=65537, key_size=2048)

    # Pripravimo certifikatni zahtevek
    builder = x509.CertificateSigningRequestBuilder()

    # Naj dodamo ime kot polje COMMON NAME
    subject_name_csr = x509.Name([x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)])
    builder = builder.subject_name(subject_name_csr)

    # Če uporabnik poda email naslov, ga zapišemo v SubjectAlternativeName
    # V formatu RFC822Name
    # Polje ni nujno (kritično)
    if email:
        pass
        #san_email = x509.SubjectAlternativeName([x509.RFC822Name(email)])
        #builder = builder.add_extension(san_email, critical=False)
    
    csr = builder.sign(private_key_csr, hashes.SHA256())

    return csr, private_key_csr


def issue_certificate(cert_ca, private_key_ca, csr):
    builder = x509.CertificateBuilder()
    builder = builder.subject_name(csr.subject)
    builder = builder.issuer_name(cert_ca.subject)
    builder = builder.public_key(csr.public_key())
    builder = builder.serial_number(x509.random_serial_number())

    for ext in csr.extensions:
        builder = builder.add_extension(ext.value, critical=False)    
    
    now = datetime.datetime.utcnow()
    builder = builder.not_valid_before(now)
    builder = builder.not_valid_after(now + datetime.timedelta(days=365))

    return builder.sign(private_key_ca, hashes.SHA256())


def save_private_key(private_key, filename):
    with open(filename, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))


def save_certificate(cert, filename):
    with open(filename, "wb") as f:
        f.write(cert.public_bytes(serialization.Encoding.PEM))    

In [60]:
cert_ca, sk_ca = create_ca("Certifikatna agencija VP")
save_certificate(cert_ca, "cert_ca.pem")
save_private_key(sk_ca, "sk_ca.pem")

csr_ana, sk_ana = create_csr("Ana", "ana@vp.si")
save_private_key(sk_ana, "sk_ana.pem")
cert_ana = issue_certificate(cert_ca, sk_ca, csr_ana)
save_certificate(cert_ana, "cert_ana.pem")

csr_bor, sk_bor = create_csr("Bor", "bor@vp.si")
save_private_key(sk_bor, "sk_bor.pem")
cert_bor = issue_certificate(cert_ca, sk_ca, csr_bor)
save_certificate(cert_bor, "cert_bor.pem")

csr_cene, sk_cene = create_csr("Cene", "cene@vp.si")
save_private_key(sk_cene, "sk_cene.pem")
cert_cene = issue_certificate(cert_ana, sk_ana, csr_cene)
save_certificate(cert_cene, "cert_cene.pem")